# Задача

Представьте, что вы работаете дата-сайентистом в компании Booking. Одна из проблем компании — это нечестные отели, которые накручивают себе рейтинг. Одним из способов обнаружения таких отелей является построение модели, которая предсказывает рейтинг отеля. Если предсказания модели сильно отличаются от фактического результата, то, возможно, отель ведёт себя нечестно, и его стоит проверить.

Вам поставлена задача создать такую модель

# Создание среды

In [6]:
import numpy as np 
import pandas as pd 

# библиотеки для визуализации
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# библиотеки для создания и обучения модели
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

# Импортируем библиотеки, необходимые для преобразования признаков
import category_encoders as ce
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import geopy

# системный библиотеки
import time
from termcolor import colored

In [7]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [8]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# Получение и обзор данных

In [9]:
# Подгрузим наши данные из соревнования

DATA_DIR = '/kaggle/input/sf-booking/'
df_train = pd.read_csv(DATA_DIR+'/hotels_train.csv') # датасет для обучения
df_test = pd.read_csv(DATA_DIR+'hotels_test.csv') # датасет для предсказания
sample_submission = pd.read_csv(DATA_DIR+'/submission.csv') # самбмишн

In [10]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               386803 non-null  object 
 1   additional_number_of_scoring                386803 non-null  int64  
 2   review_date                                 386803 non-null  object 
 3   average_score                               386803 non-null  float64
 4   hotel_name                                  386803 non-null  object 
 5   reviewer_nationality                        386803 non-null  object 
 6   negative_review                             386803 non-null  object 
 7   review_total_negative_word_counts           386803 non-null  int64  
 8   total_number_of_reviews                     386803 non-null  int64  
 9   positive_review                             386803 non-null  object 
 

In [11]:
df_train.head(2)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671
1,130 134 Southampton Row Camden London WC1B 5AF...,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097


In [12]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128935 entries, 0 to 128934
Data columns (total 16 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               128935 non-null  object 
 1   additional_number_of_scoring                128935 non-null  int64  
 2   review_date                                 128935 non-null  object 
 3   average_score                               128935 non-null  float64
 4   hotel_name                                  128935 non-null  object 
 5   reviewer_nationality                        128935 non-null  object 
 6   negative_review                             128935 non-null  object 
 7   review_total_negative_word_counts           128935 non-null  int64  
 8   total_number_of_reviews                     128935 non-null  int64  
 9   positive_review                             128935 non-null  object 
 

In [13]:
df_test.head(2)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,tags,days_since_review,lat,lng
0,Via Senigallia 6 20161 Milan Italy,904,7/21/2017,8.1,Hotel Da Vinci,United Kingdom,Would have appreciated a shop in the hotel th...,52,16670,Hotel was great clean friendly staff free bre...,62,1,"[' Leisure trip ', ' Couple ', ' Double Room '...",13 days,45.533137,9.171102
1,Arlandaweg 10 Westpoort 1043 EW Amsterdam Neth...,612,12/12/2016,8.6,Urban Lodge Hotel,Belgium,No tissue paper box was present at the room,10,5018,No Positive,0,7,"[' Leisure trip ', ' Group ', ' Triple Room ',...",234 day,52.385649,4.834443


In [14]:
sample_submission.head(2)

,reviewer_score,id
0,1,488440
1,10,274649


In [15]:
sample_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128935 entries, 0 to 128934
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype
---  ------          --------------   -----
 0   reviewer_score  128935 non-null  int64
 1   id              128935 non-null  int64
dtypes: int64(2)
memory usage: 2.0 MB


In [16]:
# Объединение датасетов в один с признаком-маяком: 1 - трейн, 0 - тест
df_train['sample'] = 1 
df_test['sample'] = 0 
df_test['reviewer_score'] = 0 # дополняем тест пизнаком reviewer_score с нулевыми значениеми
data = df_test.append(df_train, sort=False).reset_index(drop=True)

In [17]:
# Основная статистичнская информация объединённого датасета
data.describe()

,additional_number_of_scoring,average_score,review_total_negative_word_counts,total_number_of_reviews,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,lat,lng,sample,reviewer_score
count,515738.000000,515738.000000,515738.000000,515738.000000,515738.000000,515738.000000,512470.000000,512470.000000,515738.000000,515738.000000
mean,498.081836,8.397487,18.539450,2743.743944,17.776458,7.166001,49.442439,2.823803,0.749999,6.297672
std,500.538467,0.548048,29.690831,2317.464868,21.804185,11.040228,3.466325,4.579425,0.433014,3.902295
min,1.000000,5.200000,0.000000,43.000000,0.000000,1.000000,41.328376,-0.369758,0.000000,0.000000
25%,169.000000,8.100000,2.000000,1161.000000,5.000000,1.000000,48.214662,-0.143372,0.250000,0.625000
50%,341.000000,8.400000,9.000000,2134.000000,11.000000,3.000000,51.499981,0.010607,1.000000,7.900000
75%,660.000000,8.800000,23.000000,3613.000000,22.000000,8.000000,51.516288,4.834443,1.000000,9.600000
max,2682.000000,9.800000,408.000000,16670.000000,395.000000,355.000000,52.400181,16.429233,1.000000,10.000000


# Преобразовние признаков

## **Отели**

Признаков, характеризующих сам отель немного - наименование, адрес и координаты. Попробем что-то сними сделать

### Наименование отеля и его адрес

Есть гипотеза, что отели могут носить одинаковые имена, нужно это проверить и если так, то устранить. Каких-либо других полезных данных из этих признаков для целей работы получить вряд ли получится

In [18]:
# Поиск отелей с одинаковыми названиями
uniq_name = data[['hotel_address', 'hotel_name']].groupby('hotel_name').nunique()
uniq_name[uniq_name.hotel_address > 1]

,hotel_address
hotel_name,
Hotel Regina,3


In [19]:
# Создаём новый признак - город - и добоваляем его в название отеля
data['city'] = data.hotel_address.apply(lambda x: ''.join(x.split()[-5]) if x.split()[-1] == 'Kingdom' else ''.join(x.split()[-2]))
data.hotel_name = data.hotel_name + ' ' + data.city

In [20]:
# Бинарное кодирование наименований отелей (больше 15 значений)
bin_encoder = ce.BinaryEncoder()
data_bin = bin_encoder.fit_transform(data.hotel_name)
data = pd.concat([data, data_bin], axis=1)

### Расстояние до центра города

В координатах есть пропуски, их нужно устранить, так как лучшим вариантом их использования будет сгенерировать новые признаки - относительное позиционирование отеля в городе, например, относительно его центра (все города с историей, которая сконцентрирована в точке, откуда они вероятно равномерно развивались)

In [21]:
# Cоздаём новый признак из кортежей координат

data['coordinates'] = data[['lat', 'lng']].apply(lambda x: np.nan if x.isnull().any() else tuple(x), axis=1)


In [22]:
# Словарь названий отелей, у которых не заполнены координаты, и их адресов
bd = pd.isnull(data.coordinates)
nam_coord_nul = data[bd].groupby('hotel_name')['hotel_address'].max().to_dict()
nam_coord_nul

{'Austria Trend Hotel Schloss Wilhelminenberg Wien Vienna': 'Savoyenstra e 2 16 Ottakring 1160 Vienna Austria',
 'City Hotel Deutschmeister Vienna': 'Gr nentorgasse 30 09 Alsergrund 1090 Vienna Austria',
 'Cordial Theaterhotel Wien Vienna': 'Josefst dter Stra e 22 08 Josefstadt 1080 Vienna Austria',
 'Derag Livinghotel Kaiser Franz Joseph Vienna Vienna': 'Sieveringer Stra e 4 19 D bling 1190 Vienna Austria',
 'Fleming s Selection Hotel Wien City Vienna': 'Josefst dter Stra e 10 12 08 Josefstadt 1080 Vienna Austria',
 'Holiday Inn Paris Montmartre Paris': '23 Rue Damr mont 18th arr 75018 Paris France',
 'Hotel Advance Barcelona': 'Sep lveda 180 Eixample 08011 Barcelona Spain',
 'Hotel Atlanta Vienna': 'W hringer Stra e 33 35 09 Alsergrund 1090 Vienna Austria',
 'Hotel City Central Vienna': 'Taborstra e 8 A 02 Leopoldstadt 1020 Vienna Austria',
 'Hotel Daniel Vienna Vienna': 'Landstra er G rtel 5 03 Landstra e 1030 Vienna Austria',
 'Hotel Park Villa Vienna': 'Hasenauerstra e 12 19 D bli

In [23]:
def get_city_coord(arg):
    """
    Функция получение координат по топониму
    Args:
    arg (str): наименование или адрес отеля, астионим

    Returns:
    tuple: долгота и широта через запятую
    """
    geolocator = Nominatim(user_agent='Tester')
    location = geolocator.geocode(arg)
    return (location.latitude, location.longitude)

data['city_coord'] = data.city.apply(get_city_coord) 

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Milan&format=json&limit=1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f21e767e550>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))

In [ ]:
# Замена адресов в словаре с помощью библиотеки geopy, но так как в kaggle выходит исключение,
# координаты получены в другой среде. 
hotel_dict = {'Austria Trend Hotel Schloss Wilhelminenberg Wien Vienna': (48.221415,
  16.2874559),
 'City Hotel Deutschmeister Vienna': (48.2211573, 16.3620293),
 'Cordial Theaterhotel Wien Vienna': (48.2098335, 16.3475615),
 'Derag Livinghotel Kaiser Franz Joseph Vienna Vienna': (48.2462468,
  16.3387236),
 'Flemings Selection Hotel Wien City Vienna': (48.2098335, 16.3475615),
 'Holiday Inn Paris Montmartre Paris': (48.8888806, 2.3331948),
 'Hotel Advance Barcelona': (41.3832344, 2.162940206993912),
 'Hotel Atlanta Vienna': (48.2203284, 16.3558289),
 'Hotel City Central Vienna': (48.2278937, 16.3882026),
 'Hotel Daniel Vienna Vienna': (48.1888627, 16.3838516356697),
 'Hotel Park Villa Vienna': (48.2331682, 16.3470804),
 'Hotel Pension Baron am Schottentor Vienna': (48.2212841, 16.3546402),
 'Maison Albar Hotel Paris Op ra Diamond Paris': (48.875248, 2.3233746),
 'Mercure Paris Gare Montparnasse Paris': (48.839701, 2.323519),
 'NH Collection Barcelona Podium Barcelona': (41.3929354, 2.1758883),
 'Renaissance Barcelona Hotel Barcelona': (41.3927154, 2.1674921),
 'Roomz Vienna Vienna': (48.18672145, 16.420653090404993)}

# Создание промежуточного датасета с недостающими координатами
coord_new = pd.DataFrame(hotel_dict.items(), columns=['hotel_name', 'coordinates'])

# Заполнение пропусков в координатах
data = data.merge(
    coord_new,
    on='hotel_name',
    how='left',
    suffixes=('','\x00')).sort_index(axis=1).fillna(method='bfill')[data.columns]

In [ ]:
# Получение координат центров городов (в другой среде)
city_dict = {'Milan': (45.4641943, 9.1896346),
 'Amsterdam': (52.3730796, 4.8924534),
 'Barcelona': (41.3828939, 2.1774322),
 'London': (51.5073359, -0.12765),
 'Paris': (48.8588897, 2.3200410217200766),
 'Vienna': (48.2083537, 16.3725042)}

# Создание нового промежуточного датафрейма с координатами городов
city_coord = pd.DataFrame(city_dict.items(), columns=['city', 'city_coord'])

In [ ]:
# Содинение датафреймов
data = data.merge(
    city_coord,
    on='city',
    how='left'
    )

# Создание нового признака с расстоянием до центра города в км
data['distance'] = data[['coordinates', 'city_coord']].apply(lambda x: str(geodesic(x[0], x[1])).split()[0], axis=1).astype(float).round(2)

## **Постояльцы**

Все люди разные, а значит будет разная и их реакция на одно и тоже явление, событие или предмет, поэтому необходимо разделить ревьюеров на основании имеющейся о них информации: будем пытаться использовать данные о национальности, характеристики их путешествия, их активности в оценках отелей

### Национальность

В данных есть сведения лишь о странах, по-видимому, гражданами которых являются постояльцы отелей, что, конечно, не предвносит какой-то национальный колорит в оценках, но может оказать на них влияние. Таких стран более 200, поэтому возьмём только 10 тех, чьи граждане оставили максимальное количество отзывов, остальные поместим в одну группу 

In [ ]:
print('Количество стран, из которых приехали ревьюеры', colored(f'{data.reviewer_nationality.nunique()}', 'red'))

In [ ]:
# Проведём однокартное кодирование 10 национальностей постояльцев, оставивших наибольшее количество отзывов
top_nat = data['reviewer_nationality'].value_counts().nlargest(10).index
data['reviewer_nationality'] = data['reviewer_nationality'].apply(lambda x: x if x in top_nat else 'other' )
data = pd.get_dummies(data, columns=['reviewer_nationality'])

### Теги, указанные путешественниками

Ревьюерам, видимо, предлагалось указать сведения о своём посещении по шести критериям:
- путешествие с домашними животными
- состав путешествующих
- цель поездки
- характеристика размещения
- продолжительность поездки
- использование мобильного устройства для отправки отзыва

предположим, что на оценку может повлиять только состав путешественников, так как коллективный разум может существенно изменять оценку, а также цель поездки, так как в командировке менее тщательно выбираешь отель, а значит и менее предвзято к нему относишься

In [ ]:
# Посчитаем количество уникальных тегов
tage_cnt = {}
for x in data.tags:
    for i in x[2:-2].strip().split(' \', \' '):
        if i in tage_cnt:
            tage_cnt[i] +=1
        else:
            tage_cnt[i] = 1

print('Первые по наибольшему количеству 10 тегов\n', colored(f'{sorted(dict(list(tage_cnt.items())[:10]), key=tage_cnt.get)}', 'red'))

In [ ]:
def get_tags(arg):
    """Функция очищает маcсив тегов от лишних символов
    Args:
        arg (str): строка тегов
        
    Returns:
        list: список тегов без лишних символов
    
    """
    tags = []
    for tag in list(arg[2:-2].strip().split(' \', \' ')):
        tag = tag.replace(' ', '_') # Заменяем нижним подчёркиванием пробелы между словами одного тега
        tags.append(tag)
        
    return tags


data.tags = data.tags.apply(get_tags)

In [ ]:
# Проверяем наиболее полные облака тэгов
data.loc[data.tags.apply(lambda x: len(x)).sort_values(ascending=False)[:10].index].tags.values

In [ ]:
# Наиболее интересным из тегов является состав путешественников, так как оценка одного человека и 
# формирование её на основе двух, а тем более нескольких мнений должны отличаться, поэтому создаём новый признак,
# в который заносим состав, разделённый на три группы - один,пара, несколко человек
data['composition'] = data.tags.apply(lambda x: 'one' if 'Solo_traveler'in x else (
        'two' if 'Couple' in x else (
        'more' if 'Group' in x or
        'Family_with_young_children' in x or 
        'Family_with_older_children' in x or
        'Travelers_with_friends' in x else 0)))

# Кодируем новый признак
data = pd.get_dummies(data, columns=['composition'])

In [ ]:
# Создаём и кодируем признак, показывающий цель поездки - отдых или работа. При этом будем исходить,
# что если этот тег не указан, то коммандировочные просто предпочли скрыть назначение своего прибытия
data['purpose'] = data.tags.apply(lambda x: 1 if 'Leisure_trip'in x else 0)

### Количество отзывов путешественников

In [ ]:
data.total_number_of_reviews_reviewer_has_given.mean()

## **Отзывы**

Основные признаки, которые должны влиять на оценку отеля, содержат как реакцию постояльцев на посещение отеля, так и обстоятельства, пр которых эта реакция родилась, например, время года 

### Дата отзыва

In [ ]:
# Изменение формата представления даты отзыва
data['review_date'] = pd.to_datetime(data['review_date'])

# Новый признак - год отзыва
data['review_year'] = data['review_date'].dt.year

# Новые признаки - сезон отзыва
def get_seasons(month):
    """Функция преобразует дату во времена года

    Args:
        month (int): номер месяца
        
    Returns:
        str: название времени года
    
    """
    if month in list(range(3,6)):
        return 'spring'
    elif month in list(range(6,9)):
        return 'summer'
    elif month in list(range(9,12)):
        return 'autumn'
    else:
        return 'winter'



data['review_seasons'] = data['review_date'].dt.month.apply(get_seasons)

In [ ]:
# Завсисимость средней оценки от сезона
pivot_seasons = data.groupby(['review_year', 'review_seasons'])['reviewer_score'].mean()
pivot_seasons

In [ ]:
# Однократное кодирование номинальный признаков
ohe_col = ['review_year', 'review_seasons']
data = pd.get_dummies(data, columns=ohe_col)


### Тональность отзыва

С большой долей вероятности можно предположить, что основой для првильной объективной оценки отеля будет усреднение субъективных оценок, которые заключены в позитивных и негативных отзывах посетителей - то, что им не понравилось (negative) и понравилось (positive) в отеле. И прежде всего необходима предобработка (очистка и токенизация) текста

In [ ]:
# Приведём весь текст в нижний регистр и удалим пробелы в начале и конце текста
data.negative_review = data.negative_review.str.lower().str.strip()
data.positive_review = data.positive_review.str.lower().str.strip()

#### **Замечание**

Достаточное количество отзывово не содержит никакой информации (nothing, n a, none, no, absolutely nothing), такие отзыву нужно собрать воедино и как-то выделить

In [ ]:
%%time
# Приводим неинформативный текст к единообразию
variety = ['no positive', 'nothing really','no negative','nothing','n a','none','nothing at all',
               'na','can t think of anything','nil','absolutely nothing', 'nothing to complain about',
               'no', 'nothing all good', 'no complaints', '']

data.negative_review = np.where(data.negative_review.any() in variety, 'nothing', data.negative_review)
data.positive_review = np.where(data.positive_review.any() in variety, 'nothing', data.positive_review)

In [ ]:
# Токинизируем отзывы
data['neg_review_tokenized'] = data.negative_review.str.split()
data['pos_review_tokenized'] = data.positive_review.str.split()

In [ ]:
def clearing_tokens(rewiew):
    return [word for word in rewiew if word.isalpha() and word not in stop_words]

stop_words = stopwords.words('english')

data['neg_review_cleared'] = data.neg_review_tokenized.apply(clearing_tokens)
data['pos_review_cleared'] = data.pos_review_tokenized.apply(clearing_tokens)

In [ ]:
# sent_analyzer = SentimentIntensityAnalyzer()
# sent_analyzer.polarity_scores(data.negative_review[0])

In [ ]:
# nltk_sentiment_analysis(data, 'negative_review', 'n_', sent_analyzera)


## Удаление лишних признаков

In [ ]:
# убираем вспомогательные и лишние признаки
# drop_list = ['days_since_review', 'review_date', 'lat', 'lng', 'purpose']
# for i in data.select_dtypes(include=['object']).columns:
#     if i not in drop_list:
#         drop_list.append(i)
# data.drop(drop_list, axis=1, inplace=True)


# Создание и обучение модели

In [ ]:
# Делим объединённый датасет на трнировочную тестовую часть, а тренировочную на признаки и таргет
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample', 'reviewer_score'], axis=1)

y = train_data.reviewer_score.values
X = train_data.drop(['reviewer_score'], axis=1)

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тренировочных данных на обучающие и проверочные
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

In [ ]:
# Создаём и обучаем модель
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)
model.fit(X_train, y_train)

In [ ]:
# Делаем предсказание на проверочных данных и рассчитываем метрику
y_pred = model.predict(X_test)

# Функция, вычисляюща Mean Absolute Percentage Error (MAPE)
def mape_func(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape

print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))

In [ ]:
# Выводим самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

In [ ]:
# Делаем предсказание на тестовой выборке и создаём файл для submit
predict_submission = 
sample_submission['reviewer_score'] = model.predict(test_data)
sample_submission.to_csv('submission.csv', index=False)